# Отработка ошибок, связанных с запуском кода

- Если после установки необходимых библиотек возникает ошибка, то скорее всего не установлен вебдрайвер Chrome для вашей ОС. Как установить вебдрайвер можно ознакомиться по этой ссылке. [selenium-python](https://selenium-python.com/install-chromedriver-chrome?ysclid=ly2ufnhjip111135754)
- Если после установки вебдрайвера код не отрабатывает либо возникает ошибка, то требуется проверить не открыт ли в памяти браузер Chrome. Если да, то требуется его выгрузить (закрыть). Это связано с тем, что данный код отрабатывает в фоновом режиме, но при этом использует вебдрайвер Chrome который может конфликтовать с самим запущенным браузером Chrome.
- Если ошибка по-прежнему возникает, то требуется проверить или переустановить используемые библиотеки.

In [ ]:
#!pip3 install --upgrade pip
#!pip3 install fake-useragent
#!pip3 install selenium_stealth
#!pip3 install selenium
#!pip3 install pandas
#!pip3 install regex

In [ ]:
import re
import time
import random
import requests
import pandas as pd

from lxml import html
from datetime import datetime
from selenium_stealth import stealth
from fake_useragent import UserAgent
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, TimeoutException

In [ ]:
class Parser_olx:

    # инициализатор 
    def __init__(self, sleep_time_min = 30, sleep_time_max = 120, upload_wait_time = 5, parsing_stopper_min = 300, parsing_stopper_max = 600, parsing_stopper_count = 10):
        # инициализируем врмя задержки между действиями
        self.sleep_time_min = sleep_time_min
        self.sleep_time_max = sleep_time_max
        # инициализируем время ожидания загрузки данных
        self.upload_wait_time = upload_wait_time
        # инициализируем время ожидания между загрузкой блока данных
        self.parsing_stopper_min = parsing_stopper_min
        self.parsing_stopper_max = parsing_stopper_max
        # инициализируем объем блока данных между стопами
        self.parsing_stopper_count = parsing_stopper_count
    
    # функция инициализации вебдрайвера с заданными параметарми
    def webdriver(self):
        # инициализируем юзер агент для сокрытия реальных данных браузера для избежания блокировки
        ua = UserAgent(browsers='chrome')
        # берем рандомный параметр пользовательского агента
        user_agent = ua.random
        # инициализируем оъект опций драйвера
        options = Options()
        # режим работы в фоновом режиме
        options.add_argument('--headless=new')
        # присваивание выбранного пользовательского агента 
        options.add_argument(f'--user-agent={user_agent}')
        # задаем максимальный размер окна чтобы сайт не переходил на работу в режиме мобильной версии в которой элементы не подгружаются тк расположены иначе
        options.add_argument('start-maximized')
        # присваеваем окну конкретный размер
        options.add_argument('--window-size=1920,1080')
        # задаем параметр отработки графики только на процессоре
        options.add_argument('--disable-gpu')
        # задаем параметр отключение хранения временных файлов
        options.add_argument('--disable-dev-shm-usage')
        # отключение уведомлений в браузере
        options.add_argument('--disable-notifications')
        # отключаем блокировку всплывающих окон
        options.add_argument('--disable-popup-blocking')
        # отключаем режим песочницы
        options.add_argument('--no-sandbox')
        # активация использования браузеру джава скрипта
        options.add_argument('--enable-javascript')
        # отклчение детекции джава скриптом браузера как браузера под автоматическоим управлением
        options.add_argument( '--disable-blink-features=AutomationControlled')
        # добавляется экспериментальная опция, исключающая переключатель "enable-automation", чтобы предотвратить автоматическое обнаружение, что браузер управляется вебдрайвером
        options.add_experimental_option('excludeSwitches', ['enable-automation'])
        # добавляется экспериментальная опция, отключающая расширение автоматизации в браузере, что также помогает скрыть использование вебдрайвера
        options.add_experimental_option('useAutomationExtension', False)
        # инициализируем вебдрайвер с заданными ранее опциями
        driver = Chrome(options=options)                               
        # настройка селениум стелс режим для анаоноимности браузера
        stealth(driver,
                user_agent=user_agent,
                languages=['ru-RU', 'ru'],
                vendor='Google Inc.',
                platform=user_agent.split(' ')[1].strip('()'),
                webgl_vendor='Intel Inc.',
                renderer='Intel Iris OpenGL Engine',
                fix_hairline=True,
                run_on_insecure_origins=True
                )
        # замена прототипа браузера для исключения детекции браузера под автоматическим управлением
        driver.execute_cdp_cmd('Page.addScriptToEvaluateOnNewDocument', {
            'source':
            '''
            const newProto = navigator.__proto__;
            delete newProto.webdriver;
            navigator.__proto__ = newProto;
            '''
            }
        )
        # удаляем все куки если они были ранее
        driver.delete_all_cookies()
        # Установка размеров окна
        driver.set_window_size(1920, 1080)
        # центровка расположения контента
        driver.set_window_position(0, 0)
        return driver
        
    # функция проверки корректности URL
    def check_url(self, url):
        try:
            # отправляет запрос для проверки статуса url, ответ 200 страница существует
            response = requests.head(url)
            if response.status_code == 200:
                return True
            else:
                return False
        except requests.RequestException:
            return False

    # функция парсинга ссылок результатов со страницы поиска
    def page_search_parser(self, driver, url_request_page):
        # передаем адрес страницы поиска в драйвер
        driver.get(url_request_page)
        # ожидание загрузки данных
        wait = WebDriverWait(driver, self.upload_wait_time)
        # скролим страницу до конца чтобы загрузились все элементы
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        # ожидания загрузки элемента для парсинга
        wait.until(EC.visibility_of_element_located((By.XPATH, "//div[@data-testid='l-card']")))
        # забираем загруженную страницу из драйвера для поиска нужных элементов
        page = html.fromstring(driver.page_source)
        # извлекаем все элементы с указанным путем
        elements = page.xpath("//div[@data-testid='l-card']/div/div/div/a/@href")
        # возвращаем массив текстового содержимого элементов с восстановленной сылкой
        results = [('https://www.olx.uz' + str(element)) for element in elements]
        return results

    # функция определяющая номер страницы
    def url_number_page(self, url_request):
        # захват групп цифр после page=
        match = re.search(r'page=(\d+)', url_request)
        if match:
            # берет только первую группу цифр из регулярки тк остальные не относятся к номеру страницы
            return int(match.group(1))
        else:
            # если элемент page= не найден то сылка является первой
            return 1
    
    # функция генерации новых страниц запроса
    def search_parser(self, url_request):
        # проверяем страницу на факт существования
        if not self.check_url(url_request):
            print(f'Ccылка не существует: {url_request}')
            # возвращаем пустой массив
            return []
        # создаем экземпляр вебдрайвера
        driver = self.webdriver()
        # обозначаем номер первой страницы
        page = 1
        # парсинг первой страницы запроса
        url_request_array = self.page_search_parser(driver, url_request)
        # проверяем страницу на факт того что выбрана первая тсраница поиска если нет то парсим введенную страницу и останавливаем ссбор сылок
        if 'page=' in url_request:
            print(f'Задана не первая страница запроса тк URL сожержит элемет page=. Парсер собрал данные только с этой страницы а не со всего запроса: {url_request}')
            # удаляем дубликаты собранных сылок
            url_request_array_result = list(set(url_request_array))
            print(f'Парсинг страниц сылок завершен. Найдено ссылок: {len(url_request_array_result)}')
            # закрываем вебдрайвер
            driver.quit()
            # возвращаем собранные ссылки
            return url_request_array_result
        print(f'Ссылки собраны со страницы: {page}')
        # цикл парсинга без условия выхода
        while True:
            # увеличиваем счетчик с каждой итерацией
            page += 1
            # проверяем кейс которому соответствует сылка тк от этого зависит генерация новых сылок
            if 'search%' not in url_request:
                if '/?currency=' not in url_request:
                    # генерация новой сылки согласно логики выбранного кейса search нет в ссыле currency нет в ссылке
                    new_url_request = url_request + '?page=' + str(page) if url_request[-1] == '/' else url_request + '/?page=' + str(page)
                else:
                    # генерация новой сылки согласно логики выбранного кейса search нет в ссыле currency есть в ссылке
                    new_url_request = url_request[:-1] + '&page=' + str(page) if url_request[-1] == '/' else url_request + '&page=' + str(page)
            else:
                url_request_parts = url_request.split('search%')
                # генерация новой сылки согласно логики выбранного кейса search есть в ссыле currency нет или есть в ссылке
                new_url_request = url_request_parts[0] + 'page=' + str(page) + '&search%' + url_request_parts[1]
            # парсинг данных сгенерированной сылки           
            new_page_array = self.page_search_parser(driver, new_url_request)
            # условия выхода из цикла, открыта страница номер меньше чем передавалась в драйвер или 50 итераций с момента старта как защита от бесконечного цикла
            if self.url_number_page(driver.current_url) < page or page == 51:
                break
            # добавление в массив данных с новой страницы
            url_request_array.extend(new_page_array)
            print(f'Ссылки собраны со страницы: {page}')
        # удаляем дубликаты собранных сылок
        url_request_array_result = list(set(url_request_array))
        print(f'Парсинг страниц сылок завершен. Найдено уникальных ссылок: {len(url_request_array_result)}')
        # закрываем вебдрайвер
        driver.quit()
        # возвращаем собранные ссылки
        return url_request_array_result
    
    # функция сбора данных со страницы объявления тк классы на странице используют динамические css наименования и могут измениться использую пути селекторов для автотестов и XPATH пути от найденных элементотв.
    def page_parser(self, parsing_url, page):
        page_data = {}
        page_data['page_url'] = parsing_url
        try:
            page_data['user_type'] = page.xpath('//*[@data-testid="main"]/div[2]/ul/li[1]/p/span/text()')[0]
        except:
            page_data['user_type'] = None
        try:
            page_data['post_teg'] = ', '.join(page.xpath('//*[@data-testid="main"]/div[2]/ul/li/p/text()'))
        except:
            page_data['post_teg'] = None
        try:    
            page_data['description'] = re.sub(r'\s+', ' ', (''.join(page.xpath('//*[@data-testid="ad_description"]/div/text()'))).replace('\n', '').strip()).replace('\t ', '\t').replace(' \t', '\t')
        except:
            page_data['description'] = None
        try:
            page_data['id'] = page.xpath('//*[@data-cy="ad-footer-bar-section"]/span[1]/text()[2]')[0]
        except:
            page_data['id'] = None
        try:
            page_data['views'] = page.xpath('//*[@data-testid="page-view-counter"]/text()')[0].split(': ')[1]
        except:
            page_data['views'] = None
        try:
            page_data['user_name'] = page.xpath('//*[@data-testid="user-profile-link"]/div/div[2]/h4/text()')[0]
        except:
            page_data['user_name'] = None
        try:
            page_data['user_link'] = 'https://www.olx.uz' + page.xpath('//*[@data-testid="user-profile-link"]/@href')[0]
        except:
            page_data['user_link'] = None
        try:
            page_data['user_registered'] = page.xpath('//*[@data-testid="user-profile-link"]/div/div[2]/p/span/text()')[0].replace(' г.', '')
        except:
            page_data['user_registered'] = None
        try:
            page_data['user_online_date'] = page.xpath('//*[@data-testid="lastSeenBox"]/span/text()')[0].replace('Онлайн ', '').replace(' г.', '')
        except:
            page_data['user_online_date'] = None
        try:
            page_data['posted'] = page.xpath('//*[@data-cy="ad-posted-at"]/text()')[0]
        except:
            page_data['posted'] = None
        try:
            page_data['post_name'] = page.xpath('//*[@data-testid="ad_title"]/h4/text()')[0]
        except:
            page_data['post_name'] = None
        try:
            page_data['cost'] = page.xpath('//*[@data-testid="ad-price-container"]/h3/text()')[0]
        except:
            page_data['cost'] = None
        try:
            page_data['haggle'] = page.xpath('//*[@data-testid="ad-price-container"]/p/text()')[0]
        except:
            page_data['haggle'] = None
        try:
            page_data['phone'] = page.xpath('//*[@data-testid="contact-phone"]/text()')[0].replace(' ', '').replace('+', '')
        except:
            page_data['phone'] = None
        try:
            page_data['reviews'] = page.xpath('//*[@data-testid="sentiment-description"]/text()')[0]
        except:
            page_data['reviews'] = None
        try:
            page_data['rating'] = page.xpath('//*[@data-testid="sentiment-title"]/text()')[0]
        except:
            page_data['rating'] = None
        try:
            page_data['category'] = ' | '.join(page.xpath('//*[@data-testid="breadcrumb-item"]/a/text()'))
        except:
            page_data['category'] = None
        try:
            user_address_part = page.xpath('//*[@id="mainContent"]/div/div[2]/div[3]/div[2]/div[3]/div/section/div[1]/div/p[1]/span/text()')
            if user_address_part:
                page_data['user_address'] = ''.join(page.xpath('//*[@data-testid="aside"]/div[3]/div/section/div[1]/div/p/text()')) + ', ' + user_address_part[0]
            else:
                page_data['user_address'] = ''.join(page.xpath('//*[@data-testid="aside"]/div[3]/div/section/div[1]/div/p/text()'))
        except:
            page_data['user_address'] = None
        # добавляем признак собранных данных для удобства отработки ошибок
        try:
            page_data['data_parsed'] = 'phone_not_parsed' if page_data['phone'] is None and page.xpath('//*[@data-testid="ad-contact-phone"]') else None
        except:
            page_data['data_parsed'] = None
        return page_data
    
    # функция для прокликивания элементов страницы и возврата блока парсинга страницы
    def clicker(self, parsing_url):
        # проверяем URL на факт существования
        if self.check_url(parsing_url):
            # создаем экземпляр вебдрайвера
            driver = self.webdriver()
            # передаем url страницы в драйвер
            driver.get(parsing_url)
            # не явное ожидание загрузки страницы 
            driver.implicitly_wait(self.upload_wait_time)
            # расчет рандомного времени ожидания
            random_wait_time = random.uniform(self.sleep_time_min, self.sleep_time_max)
            print(f'Время ожидания: {random_wait_time}')
            # применяем ожидание 
            time.sleep(random_wait_time / 2)
            # скролим страницу до конца чтобы загрузились все элементы
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            # применяем ожидание 
            time.sleep(random_wait_time / 2)
            # скролим страницу до конца вверх чтобы загрузились все элементы с которыми будем взаимодействовать
            driver.execute_script("window.scrollTo(0, 0);")
            # ожидание загрузки данных
            wait = WebDriverWait(driver, self.upload_wait_time)
            # ждем загрузки конкретных элементов страницы после прокликивания
            try:
                wait.until(EC.visibility_of_element_located((By.XPATH, "//div[@data-testid='ad_description']")))
                wait.until(EC.visibility_of_element_located((By.XPATH, "//a[@data-testid='user-profile-link']")))
            except:
                print('не найдены элемент описания или профиля')
            # пытаемся взаимодействовать с кнопкой
            try:
                # ищем кнопку
                button = driver.find_element(By.XPATH, "//button[@data-testid='ad-contact-phone']")
                # скролим до кнопки
                driver.execute_script("arguments[0].scrollIntoView();", button)
                # нажимаем на кнопку через JS
                driver.execute_script("arguments[0].click();", button)
                print('кнопка нажата')
                # ждем появление результата нажатия
                wait.until(EC.presence_of_element_located((By.XPATH, "//a[@data-testid='contact-phone']")))
            # повторная попытка взаимодействия например если структура DOM изменилась и элемент был удален и добавлен повторно
            except StaleElementReferenceException:
                print('кнопка ищется повторно')
                # не явное ожидание загрузки страницы 
                driver.implicitly_wait(self.upload_wait_time)
                # ищем кнопку
                button = driver.find_element(By.XPATH, "//button[@data-testid='ad-contact-phone']")
                # скролим до кнопки
                driver.execute_script("arguments[0].scrollIntoView();", button)
                # нажимаем на кнопку через JS
                driver.execute_script("arguments[0].click();", button)
                print('кнопка нажата')
                # ждем появление результата нажатия
                wait.until(EC.presence_of_element_located((By.XPATH, "//a[@data-testid='contact-phone']")))
            # в случае если этемент ожидания не дожидается появления результата то вызывается сообщение об ошибке
            except TimeoutException:
                print('поймана капча')
            # в случае не нахождения кнопки вызывается сообщение об ошибке
            except NoSuchElementException:
                print('телефон не указан в объявлении')
            # передаем содержимое вебдрайвера после взаимодействия со страницей из селениума в формате html
            page = html.fromstring(driver.page_source)
            # закрываем драйвер
            driver.quit()
        else:
            # передаем пустую строку если страница не существует
            page = ''
        return page
    
    # функция парсинга по ссылке поиска объявлений или массиву ранее собранных сылок объявлений 
    def parser(self, urls):
        if isinstance(urls, str):
            # запускаем сбор массива сылок
            urls = self.search_parser(urls)
        elif isinstance(urls, list):
            # Массив уже передан в функцию, ничего не меняем
            pass
        else:
            raise TypeError("Параметр url должен быть ссылкой на страницу данных или списком ранее собранных ссылок")
        # присвоиваем номер страницы
        page_number = 0
        # присвоиваем номер стопера между блоками собранных данных
        parsing_stoper = 0
        # создаем хранилище данных
        df = []
        # цикл парсинга страниц массива
        for url_task in urls:
            # увеличиваем счетчик
            page_number += 1
            # увелисиваем стоппер
            parsing_stoper += 1
            print(f'Данные собираются со страницы: {page_number} {url_task}')
            # запуск кликера для сбора номера
            clicked_page = self.clicker(url_task)
            # запуск парсера собранных страниц
            parsed_page = self.page_parser(url_task, clicked_page)
            # сохраняем собранные данные в массив
            df.append(parsed_page)
            # проверяем не поймалили мы капчу или не превышен ли заданный лимит стоппера для вызова ожидания страницы
            if parsed_page['data_parsed'] is not None or parsing_stoper == self.parsing_stopper_count:
                # генереим случайную величину в ожидании данных в рамках заданного интерала
                random_wait_time = random.uniform(self.parsing_stopper_min, self.parsing_stopper_max)
                print(f'Пауза для исключения или снятия блокировки {random_wait_time} секунд')
                # вызываем ожидание с рассчитанным интералом
                time.sleep(random_wait_time)
                # обнуляем стопер для корректного вызова повторного ожидания
                parsing_stoper = 0              
        return df

# Работа с парсером
- Для запуска парсинга сайта [olx.uz](https://www.olx.uz/) требуется составить запрос с вызовом парсера Parser_olx().parser(url_request) в данном случае парсер соберет данные сайта со страницы на результаты запроса поиска объявлений url_request. В данном случае парсер запустит сбор ссылок на объявления со страницы адрес, которой введен и автоматически начнет собирать данные с объявлений ссылки на которые были собраны. Результатом работы будет массив данных со ссылок объявлений.
- Если url_request был передан первой страницы результата поиска, то парсер сам запустит итератор по остальным страницам и дойдет до конца результата поиска, который выдал сайт. Если url_request содержит не первую страницу, то данные будут собраны только с этой страницы поиска без запуска итератора.
- Для удобства парсер можно запускать отдельно сбор ссылок с результата запроса и отдельно сбор данных с самих ссылок объявлений. Такой подход помогает оптимизировать сбор данных, например сначала собрать только ссылки сверить с теми результатами, которые собраны ранее и исключить их из сбора данных оставив только уникальные новые ссылки данные по которым не собирались ранее и запустить сбор данных только с этих ссылок.
- Для запуска сбора только ссылок объявлений нужно использовать вызов Parser_olx().search_parser(url_request) где url_request ссылка на первую страницу результата поиска, о остальных страниц итератор соберет данные сам. Если ввести не первую страницу поиска, то будут собраны результаты только с данной страницы. Результатом работы будет массив ссылок объявлений.
- Для запуска сбора данных с объявлений нужно использовать вызов Parser_olx().parser(url_array) где url_array массив собранных ссылок объявлений. Результатом работы будет массив данных со ссылок объявлений.
- При инициализации класса парсера Parser_olx() можно задавать гиперпараметры которые влияют на скорость работы и задают временные задержки: upload_wait_time - время ожидание загрузки страницы после передачи URL адреса и максимальное время ожидания загрузки элементов на странице после взаимодействия со страницей для подгрузки данных, sleep_time_min и sleep_time_max задают интервал времени между которыми генерируется рандомный интервал ожидания для имитации действий пользователя скролов и аналога ожидания загрузки и чтения информации. parsing_stopper_min и parsing_stopper_max задают интервал времени между которыми генерируется рандомный интервал ожидания для имитации пауз между сбором определенного блока данных или если результат предыдущего сбора данных поймал капчу, parsing_stopper_count определяет количество собираемых данных между паузами. Это помогает быть более похожим на действия человека при сборе информации. Если инициализировать класс без гиперпараметров, то он будет автоматически вызван с параметрами по умолчанию: sleep_time_min = 30, sleep_time_max = 120, upload_wait_time = 5, parsing_stopper_min = 300, parsing_stopper_max = 600, parsing_stopper_count = 10.
- Если при сборе информации интересует именно номер телефона пользователя, то не рекомендуется делать слишком маленькие временные интервалы так как на сайте сработает автоматическая защита и сайт будет выдавать капчу в место отображения номера после нажатия на кнопку "Показать номер". Капча начнет отображаться ориентировочно после 14 запроса иногда раньше иногда позже. Формат отображаемой капчи ReCAPTCHA. Лучший способ ее обойти это избежать ее вызова. Если система заметит парсинг, то блокировка как правило включится приблизительно на сутки. В этом случае рекомендуется пере подключить Интернет и попробовать повторно или запустить парсер позже. Если капча вызывается слишком часто, то лучше увеличить временные интервалы через гиперпараметры sleep_time_min, sleep_time_max, upload_wait_time, parsing_stopper_min, parsing_stopper_max, parsing_stopper_count.
- Если при сборе информации будет вызвана капча, то в колонку data_parsed результирующего фрейма будет занесена запись phone_not_parsed. При необходимости можно будет отфильтровать подобные значения и запустить по ним парсинг повторно.
- Если при сборе информации номер телефона пользователя не сильно интересен, например информация интересна для автоматической фильтрации и обработки, а только потом интересен номер телефона и ссылка на объявление то временные задержки sleep_time_min, sleep_time_max, upload_wait_time, parsing_stopper_min, parsing_stopper_max, parsing_stopper_count лучше сделать минимальными для ускорения работы парсера. В таком режиме мы поймаем капчу, но это не будет проблемой так как информация по номеру телефона объявления на этапе сбора информации не будет интересна.
- Для сохранения результирующего фрейма в файл CSV требуется раскоментить строки кода #df_pd = pd.DataFrame(df) которая преобразует массив в Data Frame Pandas и #df_pd.to_csv(f"Selenium_olx_{datetime.now().strftime('%d%m%Y')}.csv") которая сохранить данные в файл с наименованием "Selenium_olx_10072024.csv" в папку с запускаемым кодом. Дата будет подставляться автоматически - текущая дата сохранения результата парсинга.
# Важно
- Перед использованием парсинга продумайте логику какие данные и для чего Вы планируете собирать и отталкивайтесь от этого устанавливая временные интервалы и задавайте релевантные объемы сбора информации возможно разумнее собирать информацию с иными паузами. Помните время влияет на полученный результат!
- У сайта есть ограничение в выдаваемых ссылках. При выдаче результата поиска сайт выдает не все результаты, а чуть больше 1000 наиболее соответствующих запросу. Крайне важно для наиболее хорошего результата сделать наиболее корректный запрос поиска с нужными настройками и фильтрами сайта. Данные настройки автоматом корректируют и добавляются в ссылку запроса. Копируйте ссылку только после формирования корректного запроса, только это обеспечит нужный Вам результат.

In [ ]:
#url_request = 'https://www.olx.uz/list/q-найди-мне-идеальное-предложение/' #case1
#url_request = 'https://www.olx.uz/transport/legkovye-avtomobili/q-красная-машина/?currency=UZS' #case2
#url_request = 'https://www.olx.uz/list/q-кофемашина-автоматическая-с-капучинатором/?search%5Bfilter_enum_state%5D%5B0%5D=new' #case3
#url_request = 'https://www.olx.uz/transport/legkovye-avtomobili/q-красная-машина/?currency=UZS&search%5Bfilter_enum_car_body%5D%5B0%5D=hatchback' #case4
#url_request = 'https://www.olx.uz/transport/legkovye-avtomobili/q-красная-машина/?currency=UZS&page=2&search%5Bfilter_enum_car_body%5D%5B0%5D=hatchback'
#url_request = 'https://www.olx.uz/nedvizhimost/kvartiry/'

# парсим данные по url страницы поиска результатов
#df = Parser_olx().parser(url_request)

# для удобства рекомендуется разделять сбор ссылок и сбор даннх со страниц ссылок так комфортнее отрабатывать ошибки или запускать ограниченный, отфильтрованный список сылок например для повторного сбора данных или сбора толко новых данных
# парсим url объявлений запроса
#url_array = Parser_olx().search_parser(url_request)
# парсинг даннх на основе переданного массива url объявлений
#df = Parser_olx().parser(url_array)

# прелбразовываем df в комфортный для работы с данными формату pandas df
#df_pd = pd.DataFrame(df)
# сохраняем фрейм в файл текущей датой
#df_pd.to_csv(f"Selenium_olx_{datetime.now().strftime('%d%m%Y')}.csv")

In [ ]:
#url_request = 'https://www.olx.uz/nedvizhimost/kvartiry/arenda-dolgosrochnaya/tashkent/?currency=UYE&search%5Bprivate_business%5D=private&search%5Border%5D=filter_float_price:asc&search%5Bfilter_float_number_of_rooms:from%5D=2&search%5Bfilter_float_number_of_rooms:to%5D=5&search%5Bfilter_enum_furnished%5D%5B0%5D=yes&search%5Bfilter_enum_comission%5D%5B0%5D=no'
#url_array = Parser_olx().search_parser(url_request)
#display(len(url_array))

In [ ]:
# читаем сохраненные сылки из файла
read_urls = pd.read_csv('Selenium_olx_array31072024.csv', header=None)
data_array = read_urls.iloc[1:, 1].tolist()
# запускаем парсинг по нужному блоку сылок
df = Parser_olx().parser(data_array[81:150])

In [ ]:
# сохраняем массив сылок в файл для удобства работы
#df_urls = pd.DataFrame(url_array)
#df_urls.to_csv(f"Selenium_olx_array{datetime.now().strftime('%d%m%Y')}.csv")
#display(len(df))
display(pd.DataFrame(df))

In [ ]:
data_array[81:150]